In [ ]:
import sklearn.model_selection
import sklearn.metrics
import lightgbm as lgb
import sklearn.ensemble
import xgboost as xgb
import gc

def cv(clf, x, y, params={}, splits=3, fit_params={}):
    kfold = sklearn.model_selection.StratifiedKFold(splits, shuffle=True)
    cv_score = sklearn.model_selection.cross_validate(clf, x, y, cv=kfold, scoring={
        'accuracy': 'accuracy',
    }, return_train_score=True, verbose=5, fit_params=fit_params)
    return cv_score

In [ ]:

params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        "metric": 'multi_logloss',
        'n_jobs': -1,
    
        #better
#         'n_estimators': 600,
#         "num_leaves": 900,
#         "subsample_for_bin": 50000,
#         'max_bin': 750,
    
        #fast
        'n_estimators': 100,
        "num_leaves": 200,
        "subsample_for_bin": 20000,
        'max_bin': 512,
    
    
        "max_depth": -1,
        'min_child_samples': 100,
        'learning_rate': 0.06,
        'boost_from_average': True,
        'min_child_weight': 1e-3,
        'reg_alpha': 2,
        'reg_lambda': 5,
        'colsample_bytree':0.7, 
}
def run_cross_validation(clf, x, y, fit_params={}):
    cv_result = cv(clf, x, y, params=params, splits=3, fit_params=fit_params)
    for scorer, score in cv_result.items():
        print('%s: %s' % (scorer, score))
        print('Average %s: %f' % (scorer, score.mean()))

In [ ]:
gc.collect()
clf = lgb.LGBMClassifier(**params)
run_cross_validation(clf, train_x, train_y, {'categorical_feature': categorical_features})

In [ ]:
def train(x, y, params={}):
    clf = lgb.LGBMClassifier(**params)
    clf.fit(x, y, categorical_feature=categorical_features)
    return clf


In [ ]:
model = train(train_x, train_y, params)

In [ ]:
pred_y = model.predict(test_x.drop('uid', axis=1))

In [ ]:
result = pd.DataFrame(test_x['uid'])
result.columns = ['id']
result['label'] = pred_y

In [ ]:
result.to_csv('submission.csv', index=False)

In [ ]:
for i in range(len(train_x.columns)):
    print(train_x.columns[i], model.feature_importances_[i])